<a href="https://colab.research.google.com/github/yoshihiroo/programming-workshop/blob/master/QC4U_2022/qc4uchapter4_cirq_Japanese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QC4U 第4回 Cirq写経翻訳
2022.10.9版

東北大学 [大関先生によるQC4U](https://altema.is.tohoku.ac.jp/QC4U/)の解説コードをもとに、理解を深めるためにCirqへの翻訳をやってみているものです。Cirq初心者ですので、正しくないコードの書き方や理解が間違っているところがあるかも知れませんがご容赦ください。説明の本文は敬意をもってほぼ丸々パクらせていただいております。（掲載については大関先生の了解を得ております。ご承諾ありがとうございます。）

[元にした2022.09.30 第4回の解説コード](https://colab.research.google.com/gist/mohzeki222/202865a9fcc45bd37dc2e124d9ab0a84/qc4uchapter4.ipynb)

# Cirqのインストール

懲りずに今回も量子コンピュータの可能性を探っていきましょう！
まずはGoogleが提供するCirqをインストール。

In [ ]:
pip install cirq

いつも利用するモジュールをまずは用意しましょう。

In [ ]:
import cirq

### 前回までの振り返り

これまでに学んだものは、H,X,Z、そして制御Zゲートでした。
それぞれの特徴は、H=重ね合わせ、Xは反転、Zは傷をつける（|1>だけ符号をマイナスにする）。
そして制御Zゲートは|11>だけ傷をつけるというものでした。
これらを組み合わせて、量子ビットの相互作用を生み出すことで、物質内で起きていることをシミュレーションすることができました。


例えば磁石の中ではスピンという磁石のかけらのようなものが、絶えず向きを変えています。
しかし隣同士のスピンの間で、向きを揃えようとする相互作用をしながら、
全体の向きを揃える傾向にあります。
それを邪魔する要素は、熱揺らぎなど環境由来のものもあれば、自分から向きを倒すという操作により、阻害することもできます。

そうした状況を模擬するような量子回路を考えてみましょう。
前回もその相互作用を作り出しましたね。
量子ビットの|0>を上向き・量子ビットの|1>を下向きのスピン（磁石のかけら）として、
それぞれの向きが平行・反平行に揃うときは確率振幅はそのままで、上下逆さまになっているときはマイナスになるような回路を作りました。


In [ ]:
qc = cirq.Circuit()
q = cirq.LineQubit.range(2)

qc.append(cirq.CNOT(q[0], q[1]))
qc.append(cirq.Z(q[1]))
qc.append(cirq.CNOT(q[0], q[1]))

回路の様子を見るときには、print、もしくはSVGCircuitでしたね。

In [ ]:
#print(qc)
from cirq.contrib.svg import SVGCircuit
SVGCircuit(qc)

どのような結果になるのかをみたければ、シミュレーションを実行しましょう。
前回も作りました自作関数をまた利用します。

In [ ]:
import numpy as np
def sim_state(qc,disp=True):
  res = sim.simulate(qc)
  if disp == True:
    print(cirq.dirac_notation(np.array(res.final_state_vector)))
  return res

今回はどの状態が結局出力されるのかという結果も気になるので、
その確率に基づくヒストグラムも表示できる関数を用意しておきましょう。

In [ ]:
def sim_state_exp(qc):
  qc.append(cirq.measure(q, key='m'))
  res = sim.run(qc, repetitions=1000)
  counts = res.histogram(key='m')
  return res

In [ ]:
sim = cirq.Simulator()
state = sim_state(qc)

In [ ]:
import matplotlib.pyplot as plt

def binary_labels(num_qubits):
    return [bin(x)[2:].zfill(num_qubits) for x in range(2 ** num_qubits)]

ans = sim_state_exp(qc)
cirq.plot_state_histogram(ans, plt.subplot(), tick_label=binary_labels(2))
plt.show()

量子回路に入力される量子ビットの初期条件は|00>ですから、そのままの結果が返ってきました。
さまざまな入力を入れた場合のことを調べるには、アダマール回路を利用すると良いですね。

In [ ]:
qc2 = cirq.Circuit()
q = cirq.LineQubit.range(2)


#２つの量子ビットを重ね合わせの状態に
qc2.append(cirq.H.on_each(q))

qc2.append(cirq.CNOT(q[0], q[1]))
qc2.append(cirq.Z(q[1]))
qc2.append(cirq.CNOT(q[0], q[1]))

In [ ]:
SVGCircuit(qc2)

In [ ]:
state2 = sim_state(qc2)

狙い通り、|00>と|11>にかかる確率振幅の符号はそのままで、|01>と|10>のように２つの量子ビットが異なる場合には、その符号は逆になりました。

物質の内部では、原子スケールのミクロな対象は量子力学に従い時間発展をしています。
そのルールはいたって簡単で、指数関数の方に、虚数とエネルギー、そして時間を掛け算するだけです。

両方のスピンが揃っている場合にエネルギーが下がり（その方がお得だとスピンは思う）、互い違いになってエネルギーが上がってしまう（その方が損だとスピンは思う）。そういう状況を考えましょう。磁石のスタンダードな模型として知られるイジング模型です。
この場合、Z回路を回転Z回路にして、その角度を時間に応じて変えることにしましょう。

In [ ]:
qc3 = cirq.Circuit()
q = cirq.LineQubit.range(2)

theta = 0.3

qc3.append(cirq.H.on_each(q))

qc3.append(cirq.CNOT(q[0], q[1]))
qc3.append(cirq.rz(theta).on(q[1]))
qc3.append(cirq.CNOT(q[0], q[1]))

In [ ]:
SVGCircuit(qc3)

In [ ]:
state3 = sim_state(qc3)

量子ビット（スピン）が揃っている場合には虚数部分には(-)、互い違いの場合には虚数部分が(+)になって違いが表れていますね。
これはエネルギーによって、確率振幅が変わり、その向きがエネルギーの符号によって異なることを示しています。

### 固有状態と量子計算

次にスピンの向きを操作することを考えましょう。
ここで登場するのがXゲート、ないしは回転Xゲートです。
Xは|0>を|1>に、または|1>を|0>に倒す作用がありました。
これをかけ続けると、|0>と|1>に絶えず向きをかえることになります。


In [ ]:
qc4 = cirq.Circuit()
q = cirq.LineQubit.range(1)

theta = 0.3

qc4.append(cirq.rx(theta).on(q[0]))

In [ ]:
SVGCircuit(qc4)

In [ ]:
state4 = sim_state(qc4)

In [ ]:
from cirq_web import BlochSphere
display(BlochSphere(state_vector=cirq.to_valid_state_vector(state4.final_state_vector)))

次第に|0>から|1>へと状態が遷移していることがわかります。
この確率振幅の変化の仕方にも特徴があります。

先ほどのように、初期条件が|00>の場合にそのまま確率振幅が保存ていました。
またアダマール回路で重ね合わせの状態にして|00>、|01>、|10>、|11>に確率振幅を持たせたとしても、実はそれぞれの大きさは変わっていません。
つまり結果の出現確率は変わらないというわけです。
その意味で状態を変えることはないというわけです。
そうした状態を固有状態と呼び、各量子回路ごとにそうした固有状態が存在します。

それに対して、回転Xゲート（Xゲートも）は、|0>の確率振幅を変化させて、その分を他の状態である|1>に移動させています。
これは回転Xゲートに対して、|0>や|1>が固有状態ではないことを示します。

量子計算では、確率振幅を傷つけること、
そして確率振幅を減らして、他の状態へ遷移させることを巧みに利用して、所望の結果を得ます。
その際に重要な概念が、固有状態かどうか、です。
|0>や|1>はZ関連の影響に対しては固有状態です。
そのため、量子ビットを動かす、確率振幅を変化させるためにはXゲートなどの別の作用が必要となります。

ちなみにXゲートは、重ね合わせの状態を固有状態に持ちます。

In [ ]:
qc5 = cirq.Circuit()
q = cirq.LineQubit.range(1)

theta = 0.3

qc5.append(cirq.H(q[0]))
qc5.append(cirq.rx(theta).on(q[0]))

In [ ]:
SVGCircuit(qc5)

In [ ]:
state5 = sim_state(qc5)

|0>+|1>という重ね合わせの状態のまとまりのまま、確率振幅を変えているだけなので、固有状態になっています。

それではこの量子状態に、回転Zゲートをかけるとどうなるでしょうか？

In [ ]:
qc6 = cirq.Circuit()
q = cirq.LineQubit.range(1)

theta = 0.3

qc6.append(cirq.H(q[0]))
qc6.append(cirq.rz(theta).on(q[0]))

In [ ]:
SVGCircuit(qc6)

In [ ]:
state6 = sim_state(qc6)

|0>と|1>のそれぞれがZゲートの固有状態になっているので確率振幅のそれぞれの大きさは変わりません。
しかし重ね合わせの状態は崩れてしまっています。
同じ係数ではないのです。
重ね合わせの状態にアダマール回路をかけると|0>に戻りました。
それを利用して、重ね合わせの状態度合いを見ることができましたね。
グローバーのアルゴリズムでもやったことですが、
ここでアダマール回路をかけてみましょう。

In [ ]:
qc6.append(cirq.H(q[0]))


In [ ]:
state6 = sim_state(qc6)

重ね合わせの状態が崩れたことが影響して、|1>の状態が飛び出してきました。
重ね合わせの状態から、回転Zゲートをかけて、崩す。
アダマール回路を経ると、重ね合わせの状態を維持している部分と崩れている部分を分離することができました。
アダマール回路は、回転Xゲートで表すと90度回転に相当します。
このことから回転Xゲートを利用して、重ね合わせの状態の部分を残し、それ以外の部分が分離させます。


In [ ]:
qc7 = cirq.Circuit()
q = cirq.LineQubit.range(1)

theta = 0.3

qc7.append(cirq.H(q[0]))
qc7.append(cirq.rz(theta).on(q[0]))
qc7.append(cirq.rx(theta).on(q[0]))

In [ ]:
SVGCircuit(qc7)

In [ ]:
state7 = sim_state(qc7)

振幅の大きさの二乗を計算してみると、|0>は0.535、|1>は0.465程度です。
少し|0>の方が出現確率が上がっています。

In [ ]:
ans = sim_state_exp(qc7)
cirq.plot_state_histogram(ans, plt.subplot(), tick_label=binary_labels(1))
plt.show()

|0>と|1>の重ね合わせの状態から、|0>の状態が取り出されています。
これはRzの作用で、|0>が絞り出されていることが伺えます。
Rxは重ね合わせの状態はそのまま、それ以外の崩れた状態を押し出す役目をしています。
Rzは重ね合わせの状態のうち|0>と|1>の振幅を崩しています。

### 量子アニーリング

さてRzは回転で、その回転角度は量子力学では、量子ビットないしはスピンのエネルギーに対応することを思い出しましょう。
上方向に向いた量子ビットは確率振幅がそのままで下方向に向いた量子ビットは確率振幅が負になりました。これを先ほどと同じように解釈して、上方向だとエネルギーが下がり（お得）、下方向だとエネルギーが上がる状況に対応していると考えましょう（損）
するとお得な状態である上向きの量子ビットの状態が徐々に出現していることがわかります。

これを利用して、エネルギーの最も低い基底状態を取り出すアルゴリズムとして、
量子アニーリングがあります。
量子アニーリングでは、
最初は回転Xゲートの作用を強く、Z関連のゲートを弱目にしておき、（重ね合わせの状態）
最後は回転Xゲートの作用を弱めて、Z関連のゲートを強めにします。


In [ ]:
qc8 = cirq.Circuit()
q = cirq.LineQubit.range(1)

theta = 0.1

qc8.append(cirq.H(q[0]))

Tall = 100
for k in range(Tall):
  qc8.append(cirq.rz(theta*k/Tall).on(q[0]))
  qc8.append(cirq.rx(theta*(1-k/Tall)).on(q[0]))

In [ ]:
SVGCircuit(qc8)

In [ ]:
state8 = sim_state(qc8)

In [ ]:
ans8 = sim_state_exp(qc8)
cirq.plot_state_histogram(ans8, plt.subplot(), tick_label=binary_labels(1))
plt.show()

うまく|0>の状態だけが大きな確率で得ることができます。
複数量子ビットにおいても同じようにできます。


In [ ]:
qc9 = cirq.Circuit()
q = cirq.LineQubit.range(2)

theta = 0.2

qc9.append(cirq.H.on_each(q))

Tall = 100
for k in range(Tall):
  qc9.append(cirq.rx(theta*(1-k/Tall)).on_each(q))

  qc9.append(cirq.CNOT(q[0], q[1]))
  qc9.append(cirq.rz(theta*k/Tall).on(q[1]))
  qc9.append(cirq.CNOT(q[0], q[1]))

In [ ]:
SVGCircuit(qc9)

In [ ]:
state9 = sim_state(qc9)

In [ ]:
ans9 = sim_state_exp(qc9)
cirq.plot_state_histogram(ans9, plt.subplot(), tick_label=binary_labels(2))
plt.show()

狙い通り、|00>または|11>が非常に高い確率で得られます。

一般のイジング模型に対して利用できるようにしておきましょう。
この|0>と|1>という簡単なものでも、その解釈を広げてみると非常に多くの応用例が思い付きます。
上と下、ではなく左右のどちらが良いのか、量子コンピュータにえらばせるなど。
理系か文系かどちらにしようか。その選択を任せるなど。
２つにひとつの選択肢だけではなく、それ以外にも多くの要素が絡み合い、どのような選択が適切なのか判断に困る。その際に、エネルギーと同じように、どちらが好ましいのかを示す数値指標がある、または設定することで、最善の選択を取り出す。
そうした目標を持った数理的な問題を組合せ最適化問題と呼びます。
量子アニーリングでは、そのような組合せ最適化問題を解くことができます。


In [ ]:
def QA(J,h,s1,s2):
  n = len(h)
  qc = cirq.Circuit()
  q = cirq.LineQubit.range(n)

  #量子ビット全体に回転Xゲート
  qc.append(cirq.rx(s1).on_each(q))

  #量子ビット全体にJとhに基づいて回転Zゲート等
  for i in range(n):
    qc.append(cirq.rz(s2*h[i]).on(q[i]))

  for i in range(n):
    for j in range(n):
      if i != j:
        qc.append(cirq.CNOT(q[i], q[j]))
        qc.append(cirq.rz(s2*J[i,j]).on(q[j]))
        qc.append(cirq.CNOT(q[i], q[j]))

  return qc

それでは適当な問題を作ってみましょう。

In [ ]:
n = 3
J = - np.ones(n**2).reshape(n,n)
h = np.zeros(n)

3つの量子ビット同士で、それぞれがペアになって相互作用するものです。
しかし|01>と|10>がエネルギーの低い（得な）状態になり、|00>と|11>がエネルギーの高い（損な）状態になるというものにします。反強磁性イジング模型というものです。
フラストレーションという状態を生み出して、どのスピンも３すくみの状態で、どれが上を向いたら良いか迷う状態となります。

理論的な量子アニーリングでは、ちょっとずつ変化をさせていくと良いので、非常に小さいdtをかけつつ、k/Tallおよび(1-k/Tall)で回転XゲートとZ関連ゲートの強弱を変化させます。

In [ ]:
#step数
Tall = 100
dt = 0.01

qc10 = cirq.Circuit()
q = cirq.LineQubit.range(n)

#重ね合わせの状態から始める
qc10.append(cirq.H.on_each(q))

for k in range(Tall):
  s1 = dt*(1 - k/Tall)
  s2 = dt*k/Tall
  Uqubo = QA(J, h, s1, s2)
  qc10.append(Uqubo,q)

In [ ]:
state10 = sim_state(qc10)

In [ ]:
def binary_labels(num_qubits):
    return [bin(x)[2:].zfill(num_qubits) for x in range(2 ** num_qubits)]

ans10 = sim_state_exp(qc10)
cirq.plot_state_histogram(ans10, plt.subplot(), tick_label=binary_labels(n))
plt.show()

これをみると特定の状態が確率が高く、選ばれる様子が見て取れますね。
|001>や|010>、|100>などどれにしたら良いか、
そして|011>、|010>、|110>なども候補として登場しています。

### QAOA（量子近似最適化アルゴリズム）

さてQAOAは、量子アニーリングの回転Xゲートと、Z関連ゲートの作用をさせる時間について、
最適化を施して、効率よく最適解を得る方法です。

その最適化の基準はエネルギーで、量子回路の出力結果から、そのエネルギーを算出することができるようになる必要があります。

前回利用した。スピンの期待値でエネルギーを計算しましょう。

In [ ]:
def ene_exp(qc,h,J,n):
  state = sim_state(qc, disp=False)

  obs = []
  for k in range(n):
    obs.append(-float(h[k])*cirq.Z(q[k]))
 
  for k in range(n):
    for l in range(n):
      if k < l:
        obs.append(-float(J[k,l])*cirq.Z(q[k]))
        obs.append(-float(J[k,l])*cirq.Z(q[l]))
      elif k > l:
        obs.append(-float(J[l,k])*cirq.Z(q[k]))
        obs.append(-float(J[l,k])*cirq.Z(q[l]))

  y = np.mean(sim.simulate_expectation_values(qc, observables=obs))
  
  return y, state

これを基準にして途中のパラメータを最適化します。

In [ ]:
def ene_func(params):
  Tall = int(len(params)/2)
  qc = cirq.Circuit()
  q = cirq.LineQubit.range(n)

  #重ね合わせの状態から始める
  qc.append(cirq.H.on_each(q))

  for k in range(Tall):
    s1 = params[k]
    s2 = params[k+Tall]
    Uqubo = QA(J, h, s1, s2)
    qc.append(Uqubo,q)
    
  ene = ene_exp(qc,h,J,n)

  return ene

In [ ]:
Tall = 2
params = np.random.rand(2*Tall)

最適化の手法は前回と同様に勾配を用いない最適化手法を利用してみましょう。

In [ ]:
from scipy.optimize import minimize
result = minimize(ene_func, params, method="COBYLA", options={"maxiter": 100})

結果得られたパラメータを取得するにはresult.xでしたね。

In [ ]:
result.fun

In [ ]:
result.x

得られたパラメータで実際に走らせてみるとどうでしょうか？

In [ ]:
params = result.x
qc11 = cirq.Circuit()
q = cirq.LineQubit.range(n)

#重ね合わせの状態から始める
qc11.append(cirq.H.on_each(q))

for k in range(Tall):
  s1 = params[k]
  s2 = params[k+Tall]
  Uqubo = QA(J, h, s1, s2)
  qc11.append(Uqubo,q)

In [ ]:
ans11 = sim_state_exp(qc11)
cirq.plot_state_histogram(ans11, plt.subplot(), tick_label=binary_labels(n))
plt.show()

うまく所望の状態を得ることのできる量子回路が得られました。
ステップ数的には量子アニーリングのシミュレーションに用いたものよりも少ないものが得られましたね。

ちなみに同じ問題を株式会社Jijの開発する量子アニーリングシミュレータのOpenJijで解かせてみるとどうでしょうか。


In [ ]:
pip install openjij

この中には量子アニーリングのシミュレーション（量子モンテカルロ法によるもの）があります。

In [ ]:
from openjij import SQASampler
sampler = SQASampler()

hとJはdict形式で入力をするために以下のように加工します。
（QAシミュレータでの相互作用の符号は物理の教科書等とは逆なので符号をマイナスしています）

In [ ]:
h_dict = {}
for i in range(n):
  h_dict[i] = h[i]

J_dict = {}
for i in range(n):
  for j in range(n):
    if i != j:
      J_dict[i,j] = - J[i,j]

実行は簡単で以下のようにhとJを投げるだけです。

In [ ]:
sampleset = sampler.sample_ising(h_dict, J_dict, num_reads=10)

In [ ]:
print(sampleset.record)

もう少し具体的な問題でそれぞれ解いてみることにしましょう。
数分割問題を扱います。
いくつかの数字があって、その数字を２つに分けます。
ただし２つの数字の合計はできるだけ等しいものとして欲しいというものです。
２つの数字の合計の差が小さければ正解ということになります。
１つ１つの数字を$n_i$として、２つのグループA,Bに分けられたときは以下のようにそれぞれの数字の合計が計算されます。

\begin{equation}
I_A = \sum_{i \in A} n_i
\end{equation}
および
\begin{equation}
I_B = \sum_{i \in B} n_i
\end{equation}

これらの差がプラスでもマイナスでも小さい方が嬉しいので、差をとったものを二乗します。
\begin{equation}
(I_A - I_B)^2 = (\sum_{i \in A} n_i
- \sum_{i \in B} n_i)^2
\end{equation}
イジング模型のように0と1または-1と+1の２値を利用して、この２つのグループに分けることを考える。Aに割り当てられたものはz_i=+1、Bに割り振られたものをz_i=-1として改めて書いてみると、次のようにこの状況を数式で書き表すことができる。
\begin{equation}
(I_A - I_B)^2 = (\sum_{i} n_i z_i )^2
\end{equation}
これの最小化問題を考えれば良い。
ちょっと展開をしてみると、
\begin{equation}
(\sum_{i} n_i z_i )(\sum_{j} n_j z_j ) = 2\sum_i n_i + \sum_{i \neq j} n_in_j z_i z_j 
\end{equation}
となってイジング模型と同じ形になることがわかる。



In [ ]:
n = 6
N = np.linspace(1,n,n)

まずは1からnまでの数字を用意する。
これらを掛け合わせてJを作る（今回はhはなしで良い）


In [ ]:
h = np.zeros(n)
J = np.zeros(n**2).reshape(n,n)

for k in range(n):
  for l in range(n):
    if k != l:
      J[k,l] = - N[k]*N[l]

まずは量子アニーリングで実行してみましょう。

In [ ]:
#step数
Tall = 200
dt = 0.01

qc12 = cirq.Circuit()
q = cirq.LineQubit.range(n)

#重ね合わせの状態から始める
qc12.append(cirq.H.on_each(q))

for k in range(Tall):
  s1 = dt*k/Tall
  s2 = dt*(1 - k/Tall)
  Uqubo = QA(J, h, s1, s2)
  qc12.append(Uqubo,q)

In [ ]:
state12 = sim_state(qc12)

In [ ]:
ans12 = sim_state_exp(qc12)
cirq.plot_state_histogram(ans12, plt.subplot(), tick_label=binary_labels(n))
plt.xticks(rotation=-90)
plt.show()

グラフですと見づらいですがいくつかの状態が強調されます。
ansを見てみると数値で確認することができます。

同じようにQAOAで実行してみましょう。

In [ ]:
def ene_func(params):
  Tall = int(len(params)/2)
  qc = cirq.Circuit()
  q = cirq.LineQubit.range(n)

  #重ね合わせの状態から始める
  qc.append(cirq.H.on_each(q))

  for k in range(Tall):
    s1 = params[k]
    s2 = params[k+Tall]
    Uqubo = QA(J, h, s1, s2)
    qc.append(Uqubo,q)
    
  ene = ene_exp(qc,h,J,n)

  return ene

In [ ]:
Tall = 10
params = np.random.rand(2*Tall)

In [ ]:
from scipy.optimize import minimize
result = minimize(ene_func, params, method="COBYLA", options={"maxiter": 100})

得られた結果を取り出したいときはresult.funとresult.xです。

In [ ]:
result.fun

In [ ]:
params = result.x
qc13 = cirq.Circuit()
q = cirq.LineQubit.range(n)

#重ね合わせの状態から始める
qc13.append(cirq.H.on_each(q))

for k in range(Tall):
  s1 = params[k]
  s2 = params[k+Tall]
  Uqubo = QA(J, h, s1, s2)
  qc13.append(Uqubo,q)

In [ ]:
ans13 = sim_state_exp(qc13)
cirq.plot_state_histogram(ans13, plt.subplot(), tick_label=binary_labels(n))
plt.xticks(rotation=90)
plt.show()

なかなか難しい問題で、合計が10と11の分割2,3,5と1,4,6とかが見つかれば正解です。
1,4,5、2,3,6などもOKです。

In [ ]:
ans13.histogram(key='m')

def sim_state_exp(qc):
  qc.append(cirq.measure(q, key='m'))
  res = sim.run(qc, repetitions=1000)
  counts = res.histogram(key='m')
  return res